In [16]:
from IPython.display import display, HTML, Image, clear_output
import time
import random
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events

In [17]:
data_consent_info = """DATA CONSENT INFORMATION:

Please read:
We wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload."""

id_instructions = """Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor
e.g. if your friend was called Charlie Brown and film star was Tom Cruise
then your unique identifer would be CBTC"""

formurl = "https://docs.google.com/forms/d/e/1FAIpQLSdhxIQOCe12dFcJfpWkrfEKxtQCOQ4JXWBSI148ccS-cO8QfA/viewform?usp=sf_link"

In [18]:
def make64 (list):
    longerlist = list
    for i in range(2):
        for j in range(len(list)):
            longerlist.append(list[j])

    return longerlist

In [19]:
#necessary variable cell

pic1 = Image("10vs9.png", width = 500)
pic2 = Image("12vs14.png", width = 500)
pic3 = Image("12vs16.png", width = 500)
pic4 = Image("12vs9.png", width = 500)
pic5 = Image("14vs12.png", width = 500)
pic6 = Image("15vs20.png", width = 500)
pic7 = Image("16vs12.png", width = 500)
pic8 = Image("16vs18.png", width = 500)
pic9 = Image("18vs16.png", width = 500)
pic10 = Image("18vs20.png", width = 500)
pic11 = Image("18vs21.png", width = 500)
pic12 = Image("20vs15.png", width = 500)
pic13 = Image("20vs18.png", width = 500)
pic14 = Image("21vs18.png", width = 500)
pic15 = Image("9vs10.png", width = 500)
pic16 = Image("9vs12.png", width = 500)

piclist = [pic1, pic2, pic3, pic4, pic5, pic6, pic7, pic8, pic9, pic10, pic11, pic12,
          pic13, pic14, pic15, pic16]
filename = ["10vs9.png", "12vs14.png", "12vs16.png", "12vs9.png", "14vs12.png", "15vs20.png", "16vs12.png", "16vs18.png",
            "18vs16.png", "18vs20.png", "18vs21.png", "20vs15.png", "20vs18.png", "21vs18.png", "9vs10.png", "9vs12.png"]
leftlist = [10, 12, 12, 12, 14, 15, 16, 16, 18, 18, 18, 20, 20, 21, 9, 9]
rightlist = [9, 14, 16, 9, 12, 10, 12, 18, 16, 20, 21, 15, 18, 18, 10, 12]
ratio = []

for i in range(len(piclist)):
    if leftlist[i] < rightlist[i]:
        ratio.append(leftlist[i]/rightlist[i])
    else:
        ratio.append(rightlist[i]/leftlist[i])

#expands the list to include 64 values in each - intended test size
piclist = make64(piclist)
filename = make64(filename)
leftlist = make64(leftlist)
rightlist = make64(rightlist)
ratio = make64(ratio)

#randomly shuffles all lists but in the same order
random.seed(1)
random.shuffle(piclist)
random.seed(1)
random.shuffle(filename)
random.seed(1)
random.shuffle(leftlist)
random.seed(1)
random.shuffle(rightlist)
random.seed(1)
random.shuffle(ratio)

In [20]:
#for the results storing
resultsdict = {
    'filename' : filename,
    'nl' : leftlist,
    'nr' : rightlist,
    'ratio' :ratio,
    'ans' : [],
    'correct': [],
    'react' : []}

#for the buttons
event_info = {
    'type': '',
    'description': '',
    'time': -1
}

In [21]:
#mega important
def register_btn_event(btn):
    event_info['type'] = "button click"
    event_info['description'] = btn.description
    event_info['time'] = time.time()
    return 

In [22]:
def wait_for_event(timeout=-1, interval=0.001, max_rate=20, allow_interupt=True):
    start_wait = time.time()

    event_info['type'] = ""
    event_info['description'] = ""
    event_info['time'] = -1

    n_proc = int(max_rate*interval)+1
    
    with ui_events() as ui_poll:
        keep_looping = True
        while keep_looping==True:
            # process UI events
            ui_poll(n_proc)

            # end loop if we have waited more than the timeout period
            if (timeout != -1) and (time.time() > start_wait + timeout):
                keep_looping = False
                
            # end loop if event has occured
            if allow_interupt==True and event_info['description']!="":
                keep_looping = False
                
            # add pause before looping
            # to check events again
            time.sleep(interval)
    
    # return event description after wait ends
    # will be set to empty string '' if no event occured
    return event_info

In [23]:
#function for a single test

def ANSsingle (images, leftlist, rightlist, main_area, btn1, btn2):    
    tally = 0

    with main_area: display(images)
    time.sleep(0.75)
    with main_area: clear_output()

    start = time.time()

    btn1.on_click(register_btn_event)
    btn2.on_click(register_btn_event)

    result = wait_for_event(timeout=3)
    
    end = time.time()
    reaction = end - start
    
    if leftlist > rightlist:
        anstrue = 'Left'
    else:
        anstrue = 'Right'
        
    if  result['description'] == anstrue and reaction < 3:
        tally += 1
        resultsdict['correct'].append(1)
        resultsdict['ans'].append(anstrue)
        resultsdict['react'].append(reaction)
    else:
        resultsdict['correct'].append(0)
        resultsdict['ans'].append(anstrue)
        resultsdict['react'].append(reaction)

    return tally, reaction

#since the input function can't be easily broken, instead of using the sleep function
#instead just make it so that if the user takes more than 3 seconds they get no points

In [24]:
#full function

def ANSblock (images, leftlist, rightlist):
    cleardict()
    resulttotal = 0
    reacttotal = 0

    top_area = widgets.Output(layout={"height":"60px"})
    main_area = widgets.Output(layout={"height":"300px"})
    bottom_area = widgets.Output(layout={"height":"60px"})

    btn1 = widgets.Button(description="Left")
    btn2 = widgets.Button(description="Right")
    panel = widgets.HBox([btn1, btn2])
   
    top_area.append_display_data(HTML("<h1>Left or Right</h1>"))
    bottom_area.append_display_data(panel)

    display(top_area)
    display(main_area)
    display(bottom_area)
    
    for i in range(len(images)):
        y = ANSsingle(images[i], leftlist[i], rightlist[i], main_area, btn1, btn2)
        resulttotal += y[0]
        reacttotal += y[1]

    clearui(top_area, main_area, bottom_area)
        
    return resulttotal, reacttotal

In [25]:
def cleardict():
    resultsdict['correct'] = []
    resultsdict['ans'] = []
    resultsdict['react'] = []
    return

In [26]:
def clearui(top, main, bot):
    with top: clear_output()
    with main: clear_output()
    with bot: clear_output()
    return

In [27]:
def send_to_google_form(bigdict, formurl):
    form_id = formurl[34:90]
    view_form_url = f'https://docs.google.com/forms/d/e/{form_id}/viewform'
    post_form_url = f'https://docs.google.com/forms/d/e/{form_id}/formResponse'

    page = requests.get(view_form_url)
    content = BeautifulSoup(page.content, "html.parser").find('script', type='text/javascript')
    content = content.text[27:-1]
    result = json.loads(content)[1][1]
    form_dict = {}
    
    loaded_all = True
    for item in result:
        if item[1] not in bigdict:
            print(f"Form item {item[1]} not found. Data not uploaded.")
            loaded_all = False
            return False
        form_dict[f'entry.{item[4][0][0]}'] = bigdict[item[1]]
    
    post_result = requests.post(post_form_url, data=form_dict)
    return post_result.ok

In [28]:
def bigdict (result, time, id):
    resultsdf = pd.DataFrame(resultsdict)
    resultsjson = resultsdf.to_json()

    finaldict = {
        'id' : id,
        'score' : result,
        'time': time,
        'resultsjson' : resultsjson}

    return finaldict

In [31]:
#Final test (run this cell) YASSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSSS

print(data_consent_info)
result = input("> ")
time.sleep(1)

if result == "yes":
    print("Thanks for your participation.")
    print("Please contact philip.lewis@ucl.ac.uk")
    print("If you have any questions or concerns")
    print("regarding the stored results.")
else:
    # end code execution by raising an exception
    raise(Exception("User did not consent to continue test."))

print(id_instructions)
user_id = input("> ")
print("User entered id:", user_id)
time.sleep(1)

HTML("<h1>Welcome to the ANS test</h1>")
time.sleep(1)
print("2 images will appear on your screen, one with blue dots and one with yellow. Please select whichever one has MORE dots. You will have 3 seconds. Type L if the left picture has more dots or R if the right picture does, then press enter")
time.sleep(3)
#print("You will be required to complete this test 3 times")
#time.sleep(2)

x = ANSblock(piclist, leftlist, rightlist)

finalresult = x[0]
finaltime = x[1]

finaldict = bigdict(finalresult, finaltime, user_id)
send_to_google_form(finaldict, formurl)

print("Thank you for completing the test!")

DATA CONSENT INFORMATION:

Please read:
We wish to record your response data
to an anonymised public data repository.
Your data will be used for educational teaching purposes
practising data analysis and visualisation.
Please type yes in the box below if you consent to the upload.


>  yes


Thanks for your participation.
Please contact philip.lewis@ucl.ac.uk
If you have any questions or concerns
regarding the stored results.
Enter your anonymised ID

To generate an anonymous 4-letter unique user identifier please enter:
- two letters based on the initials (first and last name) of a childhood friend
- two letters based on the initials (first and last name) of a favourite actor
e.g. if your friend was called Charlie Brown and film star was Tom Cruise
then your unique identifer would be CBTC


>  test


User entered id: test
2 images will appear on your screen, one with blue dots and one with yellow. Please select whichever one has MORE dots. You will have 3 seconds. Type L if the left picture has more dots or R if the right picture does, then press enter


Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython…

Output(layout=Layout(height='300px'))

Output(layout=Layout(height='60px'), outputs=({'output_type': 'display_data', 'data': {'text/plain': "HBox(chi…

Thank you for completing the test!


In [32]:
print(finaldict)

{'id': 'test', 'score': 38, 'time': 66.73136520385742, 'resultsjson': '{"filename":{"0":"16vs12.png","1":"20vs18.png","2":"14vs12.png","3":"12vs9.png","4":"12vs9.png","5":"15vs20.png","6":"20vs18.png","7":"18vs21.png","8":"12vs16.png","9":"12vs14.png","10":"18vs21.png","11":"18vs16.png","12":"21vs18.png","13":"18vs21.png","14":"18vs20.png","15":"18vs20.png","16":"20vs18.png","17":"12vs16.png","18":"9vs10.png","19":"20vs15.png","20":"9vs12.png","21":"18vs16.png","22":"9vs12.png","23":"16vs18.png","24":"18vs16.png","25":"18vs20.png","26":"10vs9.png","27":"16vs18.png","28":"12vs14.png","29":"20vs15.png","30":"15vs20.png","31":"21vs18.png","32":"18vs21.png","33":"12vs16.png","34":"12vs9.png","35":"9vs10.png","36":"14vs12.png","37":"21vs18.png","38":"15vs20.png","39":"9vs10.png","40":"9vs12.png","41":"15vs20.png","42":"10vs9.png","43":"16vs12.png","44":"20vs15.png","45":"12vs16.png","46":"12vs14.png","47":"16vs18.png","48":"16vs12.png","49":"21vs18.png","50":"18vs16.png","51":"18vs20.png","